In [ ]:
#include "DHT.h"

#define DHTPIN 2 // definisikan pin yang digunakan utk sensor DHT11
// Tentukan jenis DHT yang digunakan (pilih salah satu)
// Saat ini yang dipilih adalah DHT11 #define DHTTYPE DHT11 // DHT 11
//#define DHTTYPE DHT22 // DHT 22 (AM2302)
//#define DHTTYPE DHT21 // DHT 21 (AM2301) DHT dht(DHTPIN, DHTTYPE);
void setup() { Serial.begin(9600); Serial.println("DHTxx test!"); dht.begin();
}

void loop() {
// Baca humidity dan temperature float h = dht.readHumidity(); float t = dht.readTemperature();
// Cek hasil pembacaan, dan tampilkan bila ok if (isnan(t) || isnan(h)) {
Serial.println("Failed to read from DHT");
} else {
Serial.print("Humidity: ");//kelembaban Serial.print(h);
Serial.print(" %t"); Serial.print("Temperature: "); Serial.print(t); Serial.println(" *C");
}
}


PENGAMBILAN DATA

In [ ]:
import tkinter as tk
from tkinter import filedialog, ttk import serial
import threading import os

class DataCollector: def   init  (self):
self.temperatures = [] # Hanya menyimpan suhu self.running = False
self.serial_port = None

def start_collecting(self, port): self.serial_port = serial.Serial(port, 9600) self.running = True
threading.Thread(target=self.collect_data).start()

def collect_data(self): while self.running:
if self.serial_port.in_waiting > 0:
line = self.serial_port.readline().decode('utf-8').strip() # Ambil suhu dari output Arduino
if "Temperature:" in line:
# Ekstrak nilai suhu dari string
temperature = line.split("Temperature: ")[1].split(" *C")[0] self.temperatures.append(temperature)

def stop_collecting(self): self.running = False
if self.serial_port: self.serial_port.close()
def save_data(self, folder_name):
if not os.path.exists(folder_name): os.makedirs(folder_name)

# Menyimpan hanya suhu ke file teks
with open(os.path.join(folder_name, "temperature_data.txt"), "w") as file: for temp in self.temperatures:
file.write(temp + "\n")

class App:
def  init (self, root): self.root = root
self.root.title("Temperature Data Collector") self.root.configure(bg="#f0f0f0")

self.collector = DataCollector()
 
self.folder_name = ""

self.status_label = ttk.Label(root, text="Status: Stopped", font=("Arial", 14), background="#f0f0f0")
self.status_label.pack(pady=10)

self.start_button = ttk.Button(root, text="Start Collecting", command=self.start_collecting)
self.start_button.pack(pady=5)

self.stop_button = ttk.Button(root, text="Stop Collecting", command=self.stop_collecting)
self.stop_button.pack(pady=5)

self.save_button = ttk.Button(root, text="Save Data", command=self.save_data) self.save_button.pack(pady=5)

self.select_folder_button = ttk.Button(root, text="Select Folder", command=self.select_folder)
self.select_folder_button.pack(pady=5)

self.folder_name_label = ttk.Label(root, text="No folder selected", font=("Arial", 12), background="#f0f0f0")
self.folder_name_label.pack(pady=10)

def start_collecting(self):
port = 'COM3' # Ganti dengan port yang sesuai self.collector.start_collecting(port)
self.status_label.config(text="Status: Collecting Data", foreground="green")

def stop_collecting(self): self.collector.stop_collecting()
self.status_label.config(text="Status: Stopped", foreground="red")
def save_data(self):
if self.folder_name: self.collector.save_data(self.folder_name)
self.status_label.config(text="Data saved successfully!", foreground="blue") else:
self.status_label.config(text="Status: No folder selected", foreground="red")

def select_folder(self):
self.folder_name = filedialog.askdirectory() if self.folder_name:
self.folder_name_label.config(text=self.folder_name)

if  name	== " main ": root = tk.Tk()
app = App(root) root.mainloop()


PENGUJIAN DATA

In [ ]:
import os
import tkinter as tk
from tkinter import ttk
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
import numpy as np
import serial
import threading
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Lokasi folder data
DATA_FOLDERS = [
    "C:\\Users\\Lenovo\\Downloads\\DHT 11 PART 1\\dht11_part_1\\data\\bagus buat telur ayam",
    "C:\\Users\\Lenovo\\Downloads\\DHT 11 PART 1\\dht11_part_1\\data\\terlalu dingin buat telur ayam",
    "C:\\Users\\Lenovo\\Downloads\\DHT 11 PART 1\\dht11_part_1\\data\\terlalu panas untuk telur ayam"
]

class MachineLearningApp:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Temperature Classifier")
        self.root.geometry("400x400")
        self.serial_port = None
        self.model = None
        self.temperature_label = None

        # GUI Components
        self.status_label = ttk.Label(self.root, text="Status: Idle", font=("Arial", 14))
        self.status_label.pack(pady=10)

        ttk.Button(self.root, text="Train Model", command=self.train_model).pack(pady=5)
        ttk.Button(self.root, text="Start Real-Time Testing", command=self.start_real_time_testing).pack(pady=5)

        self.temperature_label = ttk.Label(self.root, text="Real-Time Temperature: -", font=("Arial", 16))
        self.temperature_label.pack(pady=10)

    def train_model(self):
        # Load and preprocess data
        data, labels = self.load_training_data(DATA_FOLDERS)
        data = data / 100  # Normalisasi suhu
        labels = tf.keras.utils.to_categorical(labels, num_classes=3)

        # Define the model
        self.model = Sequential([
            Dense(128, input_dim=1, activation='relu'),
            BatchNormalization(),
            Dropout(0.3),
            Dense(64, activation='relu'),
            BatchNormalization(),
            Dropout(0.3),
            Dense(32, activation='relu'),
            Dense(3, activation='softmax')
        ])
        self.model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

        # Train the model
        history = self.model.fit(data, labels, epochs=30, batch_size=8, validation_split=0.2)
        self.status_label.config(text="Model trained successfully!", foreground="green")

        # Plot training history
        plt.figure(figsize=(8, 6))
        plt.plot(history.history['accuracy'], label='Training Accuracy')
        plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
        plt.title('Model Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.show()

        # Confusion Matrix
        predicted_labels = np.argmax(self.model.predict(data), axis=1)
        true_labels = np.argmax(labels, axis=1)
        cm = confusion_matrix(true_labels, predicted_labels)
        ConfusionMatrixDisplay(cm, display_labels=["Bagus buat Telur Ayam", "Terlalu Dingin", "Terlalu Panas"]).plot()
        plt.show()

    def load_training_data(self, folders):
        data = []
        labels = []

        for label, folder in enumerate(folders):
            for file in os.listdir(folder):
                file_path = os.path.join(folder, file)
                with open(file_path, 'r') as f:
                    for line in f:
                        try:
                            temperature = float(line.strip())
                            data.append(temperature)
                            labels.append(label)
                        except ValueError:
                            continue

        return np.array(data).reshape(-1, 1), np.array(labels)

    def start_real_time_testing(self):
        port = 'COM3'  # Ganti sesuai port Arduino Anda
        self.serial_port = serial.Serial(port, 9600)
        threading.Thread(target=self.real_time_classification).start()

    def real_time_classification(self):
        while True:
            if self.serial_port.in_waiting > 0:
                line = self.serial_port.readline().decode('utf-8').strip()
                if "Temperature:" in line:
                    temperature = float(line.split("Temperature: ")[1].split(" *C")[0])
                    prediction = self.model.predict(np.array([[temperature / 100]]))
                    predicted_class = np.argmax(prediction)
                    categories = ["Bagus buat Telur Ayam", "Terlalu Dingin", "Terlalu Panas"]
                    result = categories[predicted_class]
                    self.temperature_label.config(
                        text=f"Temperature: {temperature}°C\nCategory: {result}"
                    )

if __name__ == "__main__":
    app = MachineLearningApp()
    app.root.mainloop()
